In [ ]:
import pandas as pd
from pandasai import SmartDataframe
from pandasai.llm import OpenAI

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor, export_text
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [ ]:
from sklearn.metrics import r2_score
from sklearn.model_selection import GridSearchCV

In [ ]:
!jupyter nbconvert --to webpdf openAI - Project_Final.ipynb

# 1 - Load and preview dataset

In [ ]:
house_price = pd.read_csv('Real_estate _1.csv')
house_price.head()

In [ ]:
llm = OpenAI(api_token="YOUR_API_KEY_HERE")
house_price = SmartDataframe(house_price, config ={"llm": llm})

In [ ]:
# Checking the shape of dataset.
house_price.chat('how many rows and columns are in the dataset?')

# Data cleaning and Preparation

In [ ]:
# Fixing inconsistent formatting
house_price.chat('in column names replace whitespaces to "_" and change uppercase to lowercase')

In [ ]:
# Rename 'no' column to 'transaction_id'
house_price.chat('Rename "no" to "Transaction_ID')

In [ ]:
# Checking for missing values in each column
house_price.chat('is there a missing data in the dataset?')

In [ ]:
# Checking for duplicate rows
house_price.chat('are the dublicates values in the dataset?')

In [ ]:
# Checking for data types
house_price.chat('what are the data types of values?')

# 2 - Data distribution and Outliers

In [ ]:
# Checking Data Distribution
house_price.chat('Show data distribution of continius data. show it in tables')

In [ ]:
# Graphical representation of outliers to confirm outlier detection for "x3_distance_to_the_nearest_mrt_station" and "y_house_price_of_unit_area" variables
#And checking for potential outliers in other variables
house_price.chat('display all outliers of dataset in boxplots')

In [ ]:
#Count the outliers in each of the columns
house_price.chat('count outliers in each column')

In [ ]:
house_price.chat('count outliers in each column and show it in column bars')

In [ ]:
house_price.chat('show 97th percentile of x3_distance_to_the_nearest_mrt_station column')

In [ ]:
# Replacing "x3_distance_to_the_nearest_mrt_station" outliers with 97th percentile.
house_price.chat('replace outliers in "x3_distance_to_the_nearest_mrt_station" column with 97th percentile')

In [ ]:
house_price.chat('replace outliers in "x3_distance_to_the_nearest_mrt_station" column with 4435.033050000001')

In [ ]:
#Remove records with outliers in 'y_house_price_of_unit_area' column
house_price.chat('what are the outliers of the y_house_price_of_unit_area column')

In [ ]:
# Removing 3 outliers from the "y_house_price_of_unit_area" column.
# We can see that number of rows reduced to 411.
house_price.chat('remove 3 outliers from the "y_house_price_of_unit_area" ')

# Data Analysis for Data Understanding

In [ ]:
# Display descriptive statistics of the variables
house_price.chat('Display descriptive statistics of the variables')

In [ ]:
house_price.chat('In boxplot calculate count, mean, std, min, 25% quartile, 50% quartile, 75% quartile and max values of the y_house_price_of_unit_area')

In [ ]:
# Generating and plotting the correlation matrix
house_price.chat('Generate and plot the correlation matrix')

In [ ]:
house_price.chat('is the a correlation amoung the variables?')

In [ ]:
house_price.chat('show this correlation in a table')

# 3 - Decision Tree Regression Model

# Defining best predictors and Target

In [ ]:
# Finding predictors amoung our variables.
house_price.chat('display the best predictors for the house price of unit area')

In [ ]:
# Defining the best predictors for the model.
house_price.chat('which of them are the best predictors fot the house_price_of_unit_area')

In [ ]:
# Puting x3_distance_to_the_nearest_mrt_station, x4_number_of_convenience_stores, x5_latitude, x6_longitude  under the name Best Predistors.
house_price.chat('define them as a best predictors')

In [ ]:
house_price.chat('show valuse of "y house price of unit area" column')

In [ ]:
# In my previous attempts pandasai displayed me an column of house_price_of_unit_area with 414 rows, ignoring all steps of data cleaning.

In [ ]:
# Putting House price of unit area under the name Target
#  !!!  Pandasai displays y_hous_price_of_unit_area as it was before removing outliers from this column.
house_price.chat('define this column as a Target')

# Spliting dataset into training and validation dataset.

In [ ]:
house_price.chat('crate new dataframe from target and best predictors')

In [ ]:
house_price.chat('split target and best predictors into training and validation dataset where validation dataset size i 40%')